In [26]:
filename = 'pairs_with_similarity_merged_from_server.parquet'
# filename = 'pairs_with_similarity/out_4.parquet'


# get number of rows with parquetfile
import pyarrow.parquet as pq
parquet_file = pq.ParquetFile(filename)
print('number of rows: ' + str(parquet_file.metadata.num_rows))
print(parquet_file.metadata.schema)

number of rows: 93529599
required group field_id=-1 schema {
  optional binary field_id=-1 census_id (String);
  optional binary field_id=-1 deepdao_organization_id (String);
  optional binary field_id=-1 deepdao_name (String);
  optional binary field_id=-1 census_name (String);
  optional binary field_id=-1 row_id (String);
  optional float field_id=-1 similarity;
}



In [27]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [28]:
con.execute(f'select count(*) from {filename}').df()

,count_star()
0,93529599


In [29]:
# count distinct rows
# con.execute(f'select count(distinct row_id) from {filename}').df()

In [30]:
con.execute(f'select * from {filename} limit 5').df()

,census_id,deepdao_organization_id,deepdao_name,census_name,row_id,similarity
0,30f80d86-7669-50de-83ec-ec1343a2ea54,61e9839b-f517-4141-a379-12b1f4e9085c,Tough Turtles Tribe,FstSwap,30f80d86-7669-50de-83ec-ec1343a2ea54_61e9839b-...,0.178658
1,30f80d86-7669-50de-83ec-ec1343a2ea54,015b39b2-98fe-4183-a4c9-1b9aec49b1fd,toweb3,FstSwap,30f80d86-7669-50de-83ec-ec1343a2ea54_015b39b2-...,0.208167
2,30f80d86-7669-50de-83ec-ec1343a2ea54,4fce2184-7330-411b-ad69-34bdfe5aa481,TPT DAO,FstSwap,30f80d86-7669-50de-83ec-ec1343a2ea54_4fce2184-...,0.272460
3,30f80d86-7669-50de-83ec-ec1343a2ea54,f533f01e-a196-4cb7-9303-50528c7317a2,Trips,FstSwap,30f80d86-7669-50de-83ec-ec1343a2ea54_f533f01e-...,0.159215
4,30f80d86-7669-50de-83ec-ec1343a2ea54,6107a8fa-d2e3-45ea-8619-1ab59203e093,UkraineDAO,FstSwap,30f80d86-7669-50de-83ec-ec1343a2ea54_6107a8fa-...,0.197024


In [31]:
query = f"""
SELECT
    count(distinct row_id)
FROM
    {filename}
WHERE
    similarity > 0.85
"""
con.execute(query).df()

,count(DISTINCT row_id)
0,2955


In [32]:
query = f"""
SELECT
    count(distinct row_id)
FROM
    {filename}
WHERE
    similarity == 1
"""
con.execute(query).df()

,count(DISTINCT row_id)
0,311


In [33]:
# there are 2955 potential matches of deepdao names < - > census names that
# we should review manually

In [34]:
query = f"""
SELECT
    distinct row_id,
    census_id,
    deepdao_organization_id,
    similarity,
    deepdao_name,
    census_name,
FROM
    {filename}
WHERE
    similarity > 0.85
ORDER BY
    similarity DESC
"""
over_85_similarity = con.execute(query).df()
over_85_similarity.shape

(2955, 6)

In [35]:
over_85_similarity.head()

,row_id,census_id,deepdao_organization_id,similarity,deepdao_name,census_name
0,6d7ddfc3-45c2-5d97-89a4-2eb58ab2ac89_309c23fe-...,6d7ddfc3-45c2-5d97-89a4-2eb58ab2ac89,309c23fe-7ac9-4dee-b0a4-9554f892080d,1.000001,Infinity DAO,Infinity DAO
1,4c314aee-f42d-5561-bb7d-aa973f1aaa15_4d87c6cc-...,4c314aee-f42d-5561-bb7d-aa973f1aaa15,4d87c6cc-c5da-4e0b-8e58-0b0a9415ec4c,1.000001,SIL,SIL
2,7be6f774-43d7-5918-999b-64012eabb221_d11f1261-...,7be6f774-43d7-5918-999b-64012eabb221,d11f1261-975b-49b5-a801-25af19bf941d,1.000001,Futera United,Futera United
3,4e9da6a1-25d4-59fc-b5b5-499df7bb8066_4906405c-...,4e9da6a1-25d4-59fc-b5b5-499df7bb8066,4906405c-5416-4191-b93d-8226521f81ce,1.000001,Prosper,Prosper
4,46776176-4ad2-59c6-9419-9107aeb88830_7383071b-...,46776176-4ad2-59c6-9419-9107aeb88830,7383071b-5c36-4917-8ecd-a70d2b8b42cd,1.000001,Genesis Blocks,Genesis Blocks


In [36]:
# save to excel
over_85_similarity.to_excel('over_85_similarity.xlsx')